# Data Processing (Ramirez)
Covered in this notebook:
1. Downloading final.dta from Ramirez's Google Drive.
2. Adding columns for all variables described in paper:
    a. RankDist.
    b. WikiBuzz.
    c. Implied Probability / Inverse Odds.
3. Produce a cleaned Ramirez_matches_cleaned.csv file ready for further use.

In [1]:
# Imports, Remove Warnings for notebook readability.
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=UnicodeWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
# Depends on RRS's Google Drive link.
url = 'https://drive.google.com/file/d/1ltOks54u7V6tvK22EmT5n_XL8D2V4o8R/view?usp=share_link'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

df = pd.read_stata(path)

In [3]:
df.head()

,hot_raw,year,hotness_l,lastname_l,hotness_w,lastname_w,wta,location,tournament,tier,...,wiki_min7_l,wiki_mean_player_l,wiki_med_player_l,wiki_yesterday_l,wiki_twodays_l,timezone,utc_offset,elopredict,elo_pi_hat,welo_pi_hat
0,41- Jarmila Groth,2011.0,NaN,Santonja,41.0,Groth,2.0,Brisbane,Brisbane International,International,...,NaN,NaN,NaN,NaN,NaN,Australia/Brisbane,10.0,0.637051,NaN,NaN
1,88- Arina Rodionova,2011.0,88.0,Rodionova,69.0,Jovanovski,2.0,Brisbane,Brisbane International,International,...,NaN,NaN,NaN,NaN,NaN,Australia/Brisbane,10.0,0.524215,NaN,NaN
2,72- Anastasia Pavlyuchenkova,2011.0,NaN,Kudryavtseva,72.0,Pavlyuchenkova,2.0,Brisbane,Brisbane International,International,...,NaN,NaN,NaN,NaN,NaN,Australia/Brisbane,10.0,0.710430,NaN,NaN
3,,2011.0,NaN,Kleybanova,NaN,Peers,2.0,Brisbane,Brisbane International,International,...,NaN,NaN,NaN,NaN,NaN,Australia/Brisbane,10.0,0.179002,NaN,NaN
4,66- Johanna Larsson,2011.0,66.0,Larsson,91.0,Peng,1.0,Auckland,ASB Classic,International,...,NaN,NaN,NaN,NaN,NaN,Pacific/Auckland,13.0,0.689614,NaN,NaN


## RankDist Variable

In [4]:
def get_rankdist(winner_rank: int = 0, loser_rank: int = 0) -> float:
    if np.isnan(winner_rank):
        inverse_wr = 0
    else:
        inverse_wr = 1 / winner_rank

    if np.isnan(loser_rank):
        inverse_lr = 0
    else:
        inverse_lr = 1 / loser_rank
    rankdist = -(inverse_wr - inverse_lr)
    return rankdist

df['WRankDist'] = [get_rankdist(x,y) for x,y in zip(df['wrank'], df['lrank'])]
df['LRankDist'] = [get_rankdist(y, x) for x, y in zip(df['wrank'], df['lrank'])]

## WikiBuzz Variable

In [5]:
def get_wikibuzz(winner_pageviews, winner_median, loser_pageviews, loser_median):
    try:
        buzz = np.log(winner_pageviews / winner_median) - np.log(loser_pageviews / loser_median)
    except ZeroDivisionError:
        buzz = 'ZeroDivisionError'
    return buzz

df['W_WikiBuzz'] = [get_wikibuzz(row[0], row[1], row[2], row[3]) for row in zip(df['wiki_yesterday_w'], df['wiki_med365_w'],
                                                                                df['wiki_yesterday_l'], df['wiki_med365_l'])]
df['L_WikiBuzz'] = [get_wikibuzz(row[0], row[1], row[2], row[3]) for row in zip(df['wiki_yesterday_l'], df['wiki_med365_l'],
                                                                                df['wiki_yesterday_w'], df['wiki_med365_w'])]

## Inverse Odds

In [6]:
def impliedprob(player_odds):
    return 1/player_odds

df['W_inverse_bestodds'] = [impliedprob(x) for x in df['maxw']]
df['L_inverse_bestodds'] = [impliedprob(x) for x in df['maxl']]
df['W_inverse_avgodds'] = [impliedprob(x) for x in df['avgw']]
df['L_inverse_avgodds'] = [impliedprob(x) for x in df['avgl']]
df['W_inverse_B365'] = [impliedprob(x) for x in df['b365w']]
df['L_inverse_B365'] = [impliedprob(x) for x in df['b365l']]

In [7]:
# Date Column
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
# Match index column
df['match_id'] = df.index

In [8]:
# Convert each row into 2 rows (1 for winner, 1 for loser) sharing a match_id.
df_winners = df[['match_id', 'tournament', 'winner', 'date', 'year', 'WRankDist', 'W_WikiBuzz', 'W_inverse_B365', 'W_inverse_avgodds', 'W_inverse_bestodds']].copy()
df_winners['outcome'] = 1
df_losers = df[['match_id', 'tournament', 'loser', 'date', 'year', 'LRankDist', 'L_WikiBuzz', 'L_inverse_B365', 'L_inverse_avgodds', 'L_inverse_bestodds']].copy()
df_losers['outcome'] = 0

df_winners.rename({'winner': 'player', 'WRankDist': 'rankdist', 'W_WikiBuzz': 'wikibuzz',
                   'W_inverse_B365': 'inverse_b365', 'W_inverse_avgodds': 'inverse_avg', 'W_inverse_bestodds': 'inverse_best'}, axis=1, inplace=True)
df_losers.rename({'loser': 'player', 'LRankDist': 'rankdist', 'L_WikiBuzz': 'wikibuzz',
                  'L_inverse_B365': 'inverse_b365', 'L_inverse_avgodds': 'inverse_avg', 'L_inverse_bestodds': 'inverse_best'}, axis=1, inplace=True)
df = pd.concat([df_winners, df_losers], axis=0)

In [9]:
# Remove bad rows.
df = df.loc[df["wikibuzz"] != np.inf]
df = df.loc[df["wikibuzz"] != -np.inf]
df = df.loc[df["wikibuzz"] != 'ZeroDivisionError']

df = df[df['wikibuzz'].notna()]
df = df[df['rankdist'].notna()]
df = df[df['inverse_avg'].notna()]

df.sort_values(by='date', inplace=True)

In [10]:
# Link players to their Wikipedia pages.
url = "https://drive.google.com/file/d/1PP6qoLuh43Fdkj5oVfqTKN4TWe6dcwfd/view?usp=share_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
players = pd.read_csv(path)

player_dict = dict(zip(players.odds_player, players.player))
new_player_dict = {
    'Pliskova_Ka.':'Karolína_Plíšková',
    'Williams_S.':'Serena_Williams',
    'Pliskova_Kr.':'Kristýna_Plíšková',
    'Alexandra_Krunic':'Aleksandra_Krunić',
    'Yingying_Duan':'Duan_Yingying',
    'Lin_Zhu':'Zhu_Lin_(tennis)',
    'Rodionova_Ar.':'Arina_Rodionova',
    'Carina_Witthoeft':'Carina_Witthöft',
    'Stefanie_Voegele':'Stefanie_Vögele',
    'Sílvia_Soler-Espinosa':'Sílvia_Soler_Espinosa',
    'Xinyun_Han':'Han_Xinyun',
    'Yafan_Wang':'Wang_Yafan',
    'Jana_Cepelova':'Jana_Čepelová',
    'Zhang_Shuai':'Zhang_Shuai',
    'Catherine_McNally':'Caty_McNally',
    'Viktoria_Kuzmova':'Viktória_Kužmová',
    'Barbora_Krejcikova':'Barbora_Krejčíková',
    'En-Shuo_Liang':'Liang_En-shuo',
    'Chloe_Paquet':'Chloé_Paquet',
    'Su-Wei_Hsieh':'Hsieh_Su-wei',
    'Saisai_Zheng':'Zheng_Saisai',
    'Marketa_Vondrousova':'Markéta_Vondroušová',
    'Fangzhou_Liu':'Liu_Fangzhou',
    'Patricia_Maria_Tig':'Patricia_Maria_Țig',
    'Julia_Goerges':'Julia_Görges',
    'Lesley_Pattinama_Kerkhove':'Lesley_Pattinama_Kerkhove',
    'Tamara_Zidansek':'Tamara_Zidanšek',
    'Iga_Swiatek':'Iga_Świątek',
    'Anna_Karolina_Schmiedlova':'Anna_Karolína_Schmiedlová',
    'Shuai_Peng':'Peng_Shuai',
    'Leonie_Kung':'Leonie_Küng',
    'Radwanska_U.':'Agnieszka_Radwańska',
    'Barbora_Strycova':'Barbora_Strýcová',
    'Cagla_Buyukakcay':'Çağla_Büyükakçay',
    'Rodionova_An.':'Arina_Rodionova',
    'Alexandra_Cadantu':'Alexandra_Cadanțu-Ignatik',
    'Marie_Bouzkova':'Marie_Bouzková',
    'Denisa_Allertova':'Denisa_Šátralová',
    'Lucie_Hradecka':'Lucie_Hradecká',
    'Ivana_Jorovic':'Ivana_Jorović',
    'Maia_Lumsden':'Maia_Lumsden',
    'Mirjana_Lucic':'Mirjana_Lučić-Baroni',
    'Hailey_Baptiste':'Hailey_Baptiste',
    'Katerina_Siniakova':'Kateřina_Siniaková',
    'Mihaela_Buzarnescu':'Mihaela_Buzărnescu',
    'Qiang_Wang':'Wang_Qiang_(tennis)',
    'Xiaodi_You':'You_Xiaodi',
    'Paula_Cristina_Goncalves':'Paula_Cristina_Gonçalves',
    'Aliona_Bolsova':'Aliona_Bolsova',
    'Tereza_Smitkova':'Tereza_Smitková',
    'Xinyu_Wang':'Wang_Xinyu',
    'Leylah_Fernandez':'Leylah_Fernandez',
    'Magdalena_Rybarikova':'Magdaléna_Rybáriková',
    'Johanna_Larsson':'Johanna_Larsson',
    'Mirjana_Lucic-Baroni':'Mirjana_Lučić-Baroni',
    'Danka_Kovinic':'Danka_Kovinić',
    'Tereza_Martincova':'Tereza_Martincová',
    'Montserrat_Gonzalez':'Montserrat_González',
    'Maria_Herazo_Gonzalez':'María_Herazo_González',
    'Ipek_Soylu':'İpek_Soylu',
    'Petra_Martic':'Petra_Martić',
    'Timea_Babos':'Tímea_Babos',
    'Nina_Stojanovic':'Nina_Stojanović',
    'Na-Lae_Han':'Han_Na-lae',
    'Jia-Jing_Lu':'Lu_Jiajing', #From 2nd iteration, losers column...
    'Lesley_Pattinama_Kerkhove':'Lesley_Pattinama_Kerkhove',
    'Ya-Hsuan_Lee':'Lee_Ya-hsuan',
    'Alexandra_Cadanțu-Ignatik':'Alexandra_Cadanțu-Ignatik',
    'Maria_Mateas':'Maria_Mateas',
    'Selena_Janicijevic':'Séléna_Janicijevic',
    'Jessica_Pieri':'Jessica_Pieri',
    'Denisa_Šátralová':'Denisa_Šátralová',
    'Kamilla_Rakhimova':'Kamilla_Rakhimova',
    'Tess_Sugnaux':'Tess_Sugnaux',
    'Maia_Lumsden':'Maia_Lumsden',
    'Mirjam_Bjorklund':'Mirjam_Björklund',
    'Alyssa_Mayo':'Alyssa_Mayo',
    'Komola_Umarova':'Komola_Umarova',
    'Ng_Kwan-yau':'Ng_Kwan-yau',
    'Cristiana_Ferrando':'Cristiana_Ferrando',
    'Martina_Capurro_Taborda':'Martina_Capurro_Taborda',
    'Elena-Gabriela_Ruse':'Elena-Gabriela_Ruse',
    'Freya_Christie':'Freya_Christie',
    'Jovana_Jovic':'Jovana_Jović',
    'Mira_Antonitsch':'Mira_Antonitsch',
    'Dayana_Yastremska':'Dayana_Yastremska',
    'Emiliana_Arango':'Emiliana_Arango',
    'Nadia_Echeverria_Alam':'Nadia_Echeverría_Alam',
    'Frances_Altick':'Frances_Altick',
    'Ayaka_Okuno':'Ayaka_Okuno',
    'Anastasiya_Shoshyna':'Anastasiya_Shoshyna',
    'Jesika_Maleckova':'Jesika_Malečková',
    'Wushuang_Zheng':'Zheng_Wushuang',
    'Karolina_Muchova':'Karolína_Muchová',
    'Varvara_Gracheva':'Varvara_Gracheva',
    'Baindl_K.':'Kateryna_Baindl', # third iteration here
    'Uchijima_M.':'Moyuka_Uchijima',
    'Kalieva_E.':'Elvina_Kalieva',
    'Osorio_M.':'Camila_Osorio',
    'Naito_Y.':'Yuki_Naito',
    'Lazaro_A.':'Andrea_Lázaro_García',
    'Alves_C.':'Carolina_Alves_(tennis)',
    'Schunk_N.':'Nastasja_Schunk',
    'Saigo_R.':'Rina_Saigo',
    'Ioana_Minca':'Ioana_Mincă',
    'Zheng_Q.':'Zheng_Qinwen',
    'Guth_M.':'Mara_Guth',
    'Palicova_B.':'Barbora_Palicová',
    'Shibahara_E.':'Ena_Shibahara',
    'Pliskova_Ka.':'Karolína_Plíšková',
    'Reyngold_E.':'Ekaterina_Reyngold',
    'Middendorf_J.':'Julia_Middendorf',
    'Cengiz_B.':'Berfu_Cengiz',
    'Jeong_B.':'Jeong_Bo-young',
    'Szabanin_N.':'Natália_Szabanin',
    'Stakusic_M.':'Marina_Stakusic',
    'Mandlik_E.':'Elizabeth_Mandlik',
    'Contreras_Gomez_F.':'Fernanda_Contreras_Gómez',
    'Paoletti_M.':'Matilde_Paoletti',
    'Lamens_S.':'Suzan_Lamens',
    'Sherif_M.':'Mayar_Sherif',
    'Alexandra_Krunic':'Aleksandra_Krunić',
    'Bartunkova_N.':'Nikola_Bartůňková',
    'Jones_F.':'Francesca_Jones_(tennis)',
    'Yashina_E.':'Ekaterina_Yashina',
    'Mariana_Duque-Mariño':'Mariana_Duque_Mariño',
    'Bassols_Ribera_M.':'Marina_Bassols_Ribera',
    'Fruhvirtova_L.':'Linda_Fruhvirtová',
    'Laura_Pous-Tio':'Laura_Pous_Tió',
    'Williams_S.':'Serena_Williams',
    'Serban_R.':'Raluca_Șerban',
    'Caregaro_M.':'Martina_Caregaro',
    'Gabueva_A.':'Angelina_Gabueva',
    'Stearns_P.':'Peyton_Stearns',
    'Na-Lae_Han':'Han_Na-lae',
    'Scott_K.':'Katrina_Scott',
    'Lovric_P.':'Pia_Lovrič',
    'Bandecchi_S.':'Susan_Bandecchi',
    'Preston_T.':'Taylah_Preston',
    'Bronzetti_L.':'Lucia_Bronzetti',
    'Berberovic_N.':'Nefisa_Berberović',
    'Herazo_M.':'María_Herazo_González',
    'Montgomery_R.':'Robin_Montgomery',
    'Turati_B.':'Bianca_Turati',
    'Hewitt_D.':'Dalayna_Hewitt',
    'Hanatani_N.':'Nagi_Hanatani',
    'Zueger_J.':'Joanne_Züger',
    'Noskova_L.':'Linda_Nosková',
    'Marcinko_P.':'Petra_Marčinko',
    'Kraus_S.':'Sinja_Kraus',
    'Chang_S.':'Sophie_Chang',
    'Yu_E.':'Eleana_Yu',
    'Krawczyk_D.':'Desirae_Krawczyk',
    'Krueger_A.':'Ashlyn_Krueger',
    'Prisacariu_A.':'Andreea_Prisăcariu',
    'Jimenez_V.':'Victoria_Jiménez_Kasintseva',
    'En-Shuo_Liang':'Liang_En-shuo',
    'Monnet_C.':'Carole_Monnet',
    'Radivojevic_L.':'Lola_Radivojević',
    'Ruse_E-G.':'Elena-Gabriela_Ruse',
    'Morderger_Y.':'Yana_Morderger',
    'Burillo_I.':'Irene_Burillo_Escorihuela',
    'Sebov_K.':'Katherine_Sebov',
    'Mansouri_Y.':'Yasmine_Mansouri',
    'Bejlek_S.':'Sára_Bejlek',
    'Jang_S.':'Jang_Su-jeong',
    'Wurth_T.':'Tara_Würth',
    'Yingying_Duan':'Duan_Yingying',
    'Pigato_L.':'Lisa_Pigato',
    'Back_D.':'Back_Da-yeon',
    'Glushko_L.':'Lina_Glushko',
    'Kartal_S.':'Sonay_Kartal',
    'Andreeva_E.':'Erika_Andreeva',
    'Ignatik_A.':'Alexandra_Cadanțu-Ignatik',
    'Bassols_M.':'Marina_Bassols_Ribera',
    'Cross_K.':'Kayla_Cross',
    'Yifan_Xu':'Xu_Yifan',
    'Maria_Camila_Osorio_Serrano':'Camila_Osorio',
    'Anderson_R.':'Robin_Anderson_(tennis)',
    'Radisic_N.':'Nika_Radišić',
    'Parrizas_Diaz_N.':'Nuria_Párrizas_Díaz',
    'Sun_L.':'Lulu_Sun',
    'Sakatsume_H.':'Himeno_Sakatsume',
    'Bucsa_C.':'Cristina_Bucșa',
    'Jacquemot_E.':'Elsa_Jacquemot',
    'Bektas_E.':'Emina_Bektas',
    'María-Teresa_Torró-Flor':'María_Teresa_Torró_Flor',
    'Yang_Zha.':'Yang_Zhaoxuan',
    'Nuudi_M.':'Maileen_Nuudi',
    'Okamura_K.':'Kyōka_Okamura',
    'Falkowska_W.':'Weronika_Falkowska',
    'Talaba_G.':'Gabriela_Lee',
    'Kulambayeva_Z.':'Zhibek_Kulambayeva',
    'Fruhvirtova_B.':'Linda_Fruhvirtová',
    'Falkner_Z.':'Živa_Falkner',
    'Harrison_Ca.':'Catherine_Harrison_(tennis)',
    'Brace_C.':'Cadence_Brace',
    'Radwanska_U.':'Agnieszka_Radwańska',
    'Eala_A.':'Alex_Eala',
    'Plazas_J.':'Jessica_Plazas',
    'Grey_S.':'Sarah_Beth_Grey',
    'Gervais_J.':'Julie_Gervais',
    'Raducanu_E.':'Emma_Raducanu',
    'Carle_M.':'María_Carlé',
    'Tjandramulia_O.':'Olivia_Tjandramulia',
    'Di_Sarra_F.':'Federica_Di_Sarra',
    'Olyanovskaya_V.':'Valeriia_Olianovskaia',
    'Kubka_M.':'Martyna_Kubka',
    'McNally_C.':'Caty_McNally',
    'Natalija_Kostic':'Natalija_Stevanović',
    'Baszak_W.':'Weronika_Baszak',
    'Papamichail_D.':'Despina_Papamichail',
    'Bhatia_R.':'Riya_Bhatia',
    'Chwalinska_M.':'Maja_Chwalińska',
    'Havlickova_L.':'Lucie_Havlíčková',
    'Hartono_A.':'Arianne_Hartono',
    'Hontama_M.':'Mai_Hontama',
    'Gleason_Q.':'Quinn_Gleason',
    'Burrage_J.':'Jodie_Burrage',
    'Hatouka_Y.':'Yuliya_Hatouka',
    'Pliskova_Kr.':'Kristýna_Plíšková',
    'Lys_E.':'Eva_Lys',
    'Gadecki_O.':'Olivia_Gadecki',
    'Rodionova_Ar.':'Arina_Rodionova',
    'Lee_G.':'Gabriela_Lee',
    'Malygina_E.':'Elena_Malõgina',
    'Stefanini_L.':'Lucrezia_Stefanini',
    'Sutjiadi_A.':'Aldila_Sutjiadi',
    'Jeanjean_L.':'Léolia_Jeanjean',
    'Miyazaki_Y.':'Yuriko_Miyazaki',
    'Maria_Herazo_Gonzalez':'María_Herazo_González',
    'Tkacheva_M.':'Mariia_Tkacheva',
    'Mendez_S.':'Seone_Mendez',
    'Rodionova_An.':'Arina_Rodionova',
    'Saville_D.':'Daria_Saville',
    'Mishina_D.':'Daria_Mishina',
    'Andreea_Rosca':'Andreea_Roșca',
    'Babel_O.':'Océane_Babel',
    'Ji-Hee_Choi':'Choi_Ji-hee',
    'Anshba_A.':'Amina_Anshba',
    'Brancaccio_N.':'Nuria_Brancaccio',
    'Shymanovich_I.':'Iryna_Shymanovich',
    'Mboko_V.':'Victoria_Mboko',
    'Zakharova_A.':'Maria_Zakharova',
    'Riske-Amritraj_A.':'Alison_Riske-Amritraj',
    'Gasanova_A.':'Anastasia_Gasanova',
    'Selekhmeteva_O.':'Oksana_Selekhmeteva',
    'Wagner_S.':'Stephanie_Wagner',
    'Tikhonova_A.':'Anastasia_Tikhonova_(tennis)',
    'Da_Silva_Fick_G.':'Gabriella_Da_Silva-Fick',
    'Jani_R-L.':'Réka_Luca_Jani',
    'Avanesyan_E.':'Elina_Avanesyan',
    'Silva_E.':'Eden_Silva',
    'Monroy_Y.':'Yuliana_Monroy',
    'Raducànu_E.':'Emma_Raducanu',
    'Zuger_J.':'Joanne_Züger',
    'Andreeva_M.':'Erika_Andreeva',
    'Salkova_D.':'Dominika_Šalková',
    'Parks_A.':'Alycia_Parks',
    'Snigur_D.':'Daria_Snigur'
} # Introduce new player:wikipedia_page dictionary.
player_dict = player_dict | new_player_dict

df.replace({'player': player_dict}, inplace=True)

In [11]:
df

,match_id,tournament,player,date,year,rankdist,wikibuzz,inverse_b365,inverse_avg,inverse_best,outcome
12776,12776,Wimbledon,Evgeniya Rodina,2015-07-02,2015.0,0.023432,0.0,0.250000,0.246914,0.222222,0
12785,12785,Wimbledon,Ajla Tomljanovic,2015-07-02,2015.0,0.064577,0.0,0.166667,0.181818,0.164474,0
12784,12784,Wimbledon,Svetlana Kuznetsova,2015-07-02,2015.0,-0.032537,0.0,0.819672,0.813008,0.800000,0
12783,12783,Wimbledon,Jana Cepelova,2015-07-02,2015.0,0.011399,0.0,0.285714,0.288184,0.266667,0
12782,12782,Wimbledon,Mirjana Lucic,2015-07-02,2015.0,0.031481,0.0,0.381679,0.355872,0.327869,0
...,...,...,...,...,...,...,...,...,...,...,...
4139,4139,Ladies Trophy,Ekaterina Alexandrova,2020-02-15,2020.0,0.089286,1.323982,0.500000,0.483092,0.454545,0
4139,4139,Ladies Trophy,Kiki Bertens,2020-02-15,2020.0,-0.089286,-1.323982,0.555556,0.571429,0.526316,1
4142,4142,Ladies Trophy,Maria Sakkari,2020-02-15,2020.0,-0.007619,-1.612937,0.421941,0.431034,0.408163,0
4143,4143,Ladies Trophy,Kiki Bertens,2020-02-16,2020.0,-0.085000,-2.935837,0.636943,0.641026,0.621118,1


In [12]:
df.to_csv("../data/ramirez_matches_cleaned.csv")